In [2]:
import pandas as pd
import datetime
import os
import pytz
import json
print(pd.__version__)
utc = pytz.utc
AZtz = pytz.timezone("US/Arizona")

parquetName = "fitbitHypnoDf.parquet.gzip"

globalPath = "/home/chowder/Documents/personalData/fitbit/"

exportPath = "takeout-20240717T222555Z-001/Takeout/Fitbit/Global Export Data/"

print("/home/chowder/Documents/dataExports/fitbit/takeout-20240717T222555Z-001/Takeout/Fitbit/Global Export Data")
print(globalPath + exportPath)
files = [f for f in os.listdir(globalPath + exportPath) if f.split("-")[0] == "sleep"]

print(len(files))

1.1.2
/home/chowder/Documents/dataExports/fitbit/takeout-20240717T222555Z-001/Takeout/Fitbit/Global Export Data
/home/chowder/Documents/personalData/fitbit/takeout-20240717T222555Z-001/Takeout/Fitbit/Global Export Data/


FileNotFoundError: [Errno 2] No such file or directory: '/home/chowder/Documents/personalData/fitbit/takeout-20240717T222555Z-001/Takeout/Fitbit/Global Export Data/'

In [3]:
def fitbitSleepStageToStandard(stageName):
    if stageName == "restless":
        return 1
    if stageName == "asleep":
        return 2
    if stageName == "awake":
        return 0
    if stageName == "wake":
        return 0
    if stageName == "light":
        return 1
    if stageName == "deep":
        return 2
    if stageName == "rem":
        return 3
    print(stageName)
    return -1


In [32]:
data = []

for sf in files:
  with open(globalPath + exportPath + sf) as f:
    jsonData = json.load(f)
  
  for sleepSession in jsonData:
        #print(sleepSession['levels']['data'])
        #print(sleepSession)
        
        for sleepStage in sleepSession['levels']['data']:
            row = []
            stageTime = AZtz.localize(pd.to_datetime(sleepStage['dateTime'], format='%Y-%m-%dT%H:%M:%S.%f'))
            row.append(stageTime)
            row.append(stageTime + datetime.timedelta(seconds=int(sleepStage['seconds'])))
            row.append(fitbitSleepStageToStandard(sleepStage['level']))
            data.append(row)
  
print(len(data))

unknown
44712


In [33]:
data = sorted(data, key=lambda x:x[0])
data[:2]

[[Timestamp('2020-05-13 13:43:00-0700', tz='US/Arizona'),
  Timestamp('2020-05-13 13:45:00-0700', tz='US/Arizona'),
  2],
 [Timestamp('2020-05-13 13:45:00-0700', tz='US/Arizona'),
  Timestamp('2020-05-13 14:41:00-0700', tz='US/Arizona'),
  3]]

In [34]:
# add in the no data segments
filledRecords = []
# for every row except the last
for ri in range(len(data)-1):
    filledRecords.append(data[ri])
    # if there is a gap between this rows end date and the next rows start date
    if data[ri][1] < data[ri+1][0]:
        filledRecords.append([
            data[ri][1],
            data[ri+1][0],
            -1
        ])

print(len(filledRecords))

46850


In [50]:
set([type(x[1]) for x in filledRecords])

{pandas._libs.tslibs.timestamps.Timestamp}

In [37]:
ColumnNames = ["startDate", "endDate", "value"]


fitBitHypnoDf = pd.DataFrame(columns=ColumnNames, data=filledRecords)

fitBitHypnoDf = fitBitHypnoDf.set_index("startDate")
fitBitHypnoDf

,endDate,value
startDate,,
2020-05-13 13:43:00-07:00,2020-05-13 13:45:00-07:00,2.0
2020-05-13 13:45:00-07:00,2020-05-13 14:41:00-07:00,3.0
2020-05-13 14:41:00-07:00,2020-05-13 14:42:00-07:00,2.0
2020-05-13 14:42:00-07:00,2020-05-13 14:43:00-07:00,3.0
2020-05-13 14:43:00-07:00,2020-05-14 01:21:00-07:00,-1.0
...,...,...
2024-07-17 06:26:30-07:00,2024-07-17 07:51:30-07:00,2.0
2024-07-17 07:51:30-07:00,2024-07-17 07:55:30-07:00,0.0
2024-07-17 07:55:30-07:00,2024-07-17 08:22:00-07:00,2.0


In [39]:
fitBitHypnoDf.dtypes

endDate    datetime64[ns, US/Arizona]
value                         float64
dtype: object

In [49]:
fitBitHypnoDf.to_parquet(globalPath + parquetName,
              compression='gzip') 